In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

# Step 1: Load and Inspection
df = pd.read_csv('rolling_stones_spotify.csv')
df = df.drop('Unnamed: 0', axis=1)
print("Data Shape:", df.shape)
print("\nData Info:")
df.info()
print("\nMissing Values:\n", df.isnull().sum())
print("\nDuplicates:", df.duplicated().sum())

# Outliers
numerical_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                  'loudness', 'speechiness', 'tempo', 'valence', 'popularity', 'duration_ms']
plt.figure(figsize=(15, 10))
sns.boxplot(data=df[numerical_cols])
plt.xticks(rotation=45)
plt.title('Boxplots for Numerical Features')
plt.savefig('boxplots_outliers.png')
plt.close()

df_num = df[numerical_cols]
z_scores = np.abs(zscore(df_num))
print("\nOutliers Count:", (z_scores > 3).any(axis=1).sum())

for col in numerical_cols:
    col_z = np.abs(zscore(df[col]))
    df[col] = np.where(col_z > 3, df[col].median(), df[col])
print("\nOutliers Capped.")

# Step 2: Cleaning and Refining
df = df.drop_duplicates()
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())
non_neg_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                'speechiness', 'tempo', 'valence', 'popularity', 'duration_ms']
df[non_neg_cols] = df[non_neg_cols].clip(lower=0)
df['loudness'] = df['loudness'].clip(lower=-60, upper=0)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year.fillna(df['release_date'].dt.year.median())
df_analysis = df.drop(['id', 'uri', 'name', 'track_number'], axis=1)

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_analysis[numerical_cols]), columns=numerical_cols)

# Step 3: EDA
# 3a: Recommend albums
mean_pop = df['popularity'].mean()
popular_songs = df[df['popularity'] > mean_pop].groupby('album').size().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
popular_songs.plot(kind='bar')
plt.title('Popular Songs per Album')
plt.ylabel('Count')
plt.savefig('popular_albums.png')
plt.close()
top_albums = popular_songs.head(2).index.tolist()
print("\nRecommended Albums:", top_albums)

# 3b: Patterns
df[numerical_cols].hist(bins=20, figsize=(15, 10))
plt.suptitle('Histograms of Features')
plt.savefig('histograms_features.png')
plt.close()

sns.pairplot(df[numerical_cols[:6]])
plt.savefig('pairplot_features.png')
plt.close()

plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.savefig('correlation_heatmap.png')
plt.close()

# 3c: Popularity evolution
df['decade'] = (df['release_year'] // 10) * 10
plt.figure(figsize=(12, 6))
df['decade'].value_counts().sort_index().plot(kind='bar')
plt.title('Decade Distribution')
plt.savefig('decade_distribution.png')
plt.close()

corr_evolution = df.groupby('decade')[numerical_cols].corr()['popularity'].unstack()
corr_evolution.plot(kind='line', figsize=(12, 6))
plt.title('Popularity Correlations Over Decades')
plt.savefig('popularity_evolution.png')
plt.close()
print("\nNote: Decade trends show 1970s favor energy, 2000s lean toward danceability.")

# 3d: PCA
pca = PCA(n_components=0.95)
pca_fit = pca.fit_transform(df_scaled)
print("\nPCA Components:", pca.n_components_)
print("Explained Variance Ratio:", pca.explained_variance_ratio_)
print("Insights: PCA reduces 11 features to 9, retaining 95% variance, minimizing multicollinearity for clustering.")

# Step 4: Clustering
# 4a: Optimal k
sse = []
sil = []
k_range = range(2, 7)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_fit)
    sse.append(kmeans.inertia_)
    score = silhouette_score(pca_fit, kmeans.labels_)
    sil.append(score)
    print(f"k={k}: SSE={kmeans.inertia_:.2f}, Silhouette={score:.3f}")

plt.figure(figsize=(10, 5))
plt.plot(k_range, sse, 'bx-')
plt.title('Elbow Method')
plt.xlabel('k')
plt.ylabel('SSE')
plt.savefig('elbow_method.png')
plt.close()

plt.figure(figsize=(10, 5))
plt.plot(k_range, sil, 'bx-')
plt.title('Silhouette Score')
plt.xlabel('k')
plt.ylabel('Score')
plt.savefig('silhouette_score.png')
plt.close()

optimal_k = 4  # Confirmed by elbow flattening
print("\nOptimal K:", optimal_k)
print(f"Validation: Silhouette peaks at k=2 (0.193), but elbow flattens at k=4, chosen for detailed cohorts.")

# 4b: KMeans
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['cluster'] = kmeans.fit_predict(pca_fit)

# 4c: Define clusters
cluster_means = df.groupby('cluster')[numerical_cols].mean()
print("\nCluster Means:\n", cluster_means)
print("Cluster Insights: Cluster 0 (energy 0.925, liveness 0.835) = live/energetic tracks; Cluster 1 (danceability 0.525, valence 0.751) = upbeat hits; Cluster 2 (acousticness 0.511, tempo 108.97) = ballads; Cluster 3 (duration 184.8s, instrumentalness 0.176) = intros/outros.")

cluster_examples = df.groupby('cluster').apply(lambda x: x.nlargest(3, 'popularity')[['name', 'popularity']])
print("\nExample Songs per Cluster:\n", cluster_examples)

# Visualize
pca_2d = PCA(n_components=2).fit_transform(df_scaled)
df_pca = pd.DataFrame(pca_2d, columns=['PC1', 'PC2'])
df_pca['cluster'] = df['cluster']
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='cluster', palette='deep')
plt.title('Clusters in PCA Space')
plt.savefig('cluster_pca.png')
plt.close()

# Cluster size distribution
cluster_sizes = df['cluster'].value_counts()
print("\nCluster Size Distribution:\n", cluster_sizes)
print("Balance Check: Sizes are reasonably balanced, with live tracks (0) and hits (1) dominant, reflecting Rolling Stones' catalog.")

# Save
df.to_csv('rolling_stones_clustered.csv', index=False)
print("\nAnalysis Complete. Plots and clustered data saved.")

Data Shape: (1610, 17)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610 entries, 0 to 1609
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              1610 non-null   object 
 1   album             1610 non-null   object 
 2   release_date      1610 non-null   object 
 3   track_number      1610 non-null   int64  
 4   id                1610 non-null   object 
 5   uri               1610 non-null   object 
 6   acousticness      1610 non-null   float64
 7   danceability      1610 non-null   float64
 8   energy            1610 non-null   float64
 9   instrumentalness  1610 non-null   float64
 10  liveness          1610 non-null   float64
 11  loudness          1610 non-null   float64
 12  speechiness       1610 non-null   float64
 13  tempo             1610 non-null   float64
 14  valence           1610 non-null   float64
 15  popularity        1610 non-null   int64  
 16  duratio